In [1]:
import sys
sys.path.append("../../../")

In [11]:
from src.pulsed_power_ml.model_framework.training_data_labelling import trainingdata_switch_detector
from src.pulsed_power_ml.model_framework.data_io import read_training_files
from src.pulsed_power_ml.models.gupta_model.gupta_utils import read_parameters
from src.pulsed_power_ml.model_framework.training_data_labelling import get_switch_features
from src.pulsed_power_ml.model_framework.training_data_labelling import make_labeled_training_data
from src.pulsed_power_ml.model_framework.training_data_labelling import explode_to_complete_label_vector

In [16]:
RAW_DATA_BASE_FOLDER = "../../../../data/raw_data/2022-11-16_training_data/"
APPARENT_POWER_BINARY_PREFIX = "FFTApparentPower_"
APPARENT_POWER_BINARY_POSTFIX = "OffOn_FFTSize131072"
PARAMETER_FILE = "../models/gupta_model/parameters_appliance_types.yml"

# Test field

In [4]:
parameter_dict = read_parameters("../models/gupta_model/parameters_appliance_types.yml")

In [5]:
appliance_folder = "r1"
apparent_power_binary = f"{APPARENT_POWER_BINARY_PREFIX}{appliance_folder.upper()}{APPARENT_POWER_BINARY_POSTFIX}"

In [6]:
array_of_data_points = read_training_files(path_to_folder=f"{RAW_DATA_BASE_FOLDER}/{appliance_folder}",
                                           fft_size=2**17)
array_of_apparent_power_spectra = array_of_data_points[:,2*2*16:3*2**16]

In [7]:
switch_positions = trainingdata_switch_detector(spectra=array_of_apparent_power_spectra,
                                                parameters=parameter_dict)

In [8]:
features = get_switch_features(spectra=array_of_apparent_power_spectra,
                               switch_positions=switch_positions,
                               parameters=parameter_dict)

/home/thomas/.venv/py10_kitchensink/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


In [13]:
features, temp_labels = make_labeled_training_data(
    training_file=f"{RAW_DATA_BASE_FOLDER}/{appliance_folder}/{apparent_power_binary}",
    parameter_file="../models/gupta_model/parameters_appliance_types.yml",
    percentage=1.0
)

/home/thomas/.venv/py10_kitchensink/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


In [14]:
labels = explode_to_complete_label_vector(
    labels=temp_labels,
    appliance_id=7,
    parameter_file="../models/gupta_model/parameters_appliance_types.yml"
)

In [15]:
labels

array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1.

# Loop to create complete set of training data

In [20]:
iter_tuple = (
    (1, "e1"),
    (2, "f1"),
    (2, "f2"),
    (3, "fp1"),
    (4, "h1"),
    (4, "h2"),
    (5, "l1"),
    (5, "l2"),
    (6, "ps1"),
    (7, "r1")
)

In [22]:
feature_list = list()
label_list = list()
for percentage in [1.0]:
    for appliance_id, appliance_folder in iter_tuple:
        apparent_power_binary = f"{APPARENT_POWER_BINARY_PREFIX}{appliance_folder.upper()}{APPARENT_POWER_BINARY_POSTFIX}"
        features, temp_labels = make_labeled_training_data(
            training_file=f"{RAW_DATA_BASE_FOLDER}/{appliance_folder}/{apparent_power_binary}",
            parameter_file=PARAMETER_FILE,
            percentage=percentage
        )

        labels = explode_to_complete_label_vector(
            labels=temp_labels,
            appliance_id=appliance_id,
            parameter_file=PARAMETER_FILE
        )

        feature_list.append(features)
        label_list.append(labels)

/home/thomas/.venv/py10_kitchensink/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


ValueError: cannot reshape array of size 452804603 into shape (131072)